In [1]:
import twitter
import sys
import pymongo
from collections import Counter
from prettytable import PrettyTable
import numpy as np
import pandas as pd
import re
import csv

In [2]:
# SEARCHER
# credentials: "coffee1"
########################################################
def twitter_search(HASHTAG, len_results):
    
    CONSUMER_KEY = 'cOExq4T3uYu9CbdqmMUukGVqV'
    CONSUMER_SECRET = 'u9WCLUaWCLyILtHNGDAMmowxoQseQenyW0avsXhuz8NkPgLHMv'
    OAUTH_TOKEN = '4061210361-uHZhAHfTim8nnyyCWeZXMfLbypbq3Z2jgGcTmhu'
    OAUTH_TOKEN_SECRET = 'nbC4YzeS7vBBBcOWyYgr4msKdLgfRznFQAon1ji2lJn2F'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
        CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)

    print >> sys.stderr, 'Succesfully connected to Twitter SEARCH API ... ... '
    print >> sys.stderr, 'Searching for \'%s\' ... ... ' %HASHTAG
    
    max_results= min(5000, len_results)
    
    search_results = twitter_api.search.tweets(q=HASHTAG, count=100, language='en')
    
    statuses = search_results['statuses']
    
    for _ in range(179):
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError:
            print "Found \t %i \t statuses." %len(statuses) 
            break
            
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        kwargs['language'] = 'en'
        
        search_results = twitter_api.search.tweets(**kwargs)

        statuses.extend(search_results['statuses'])
        
        if len(statuses) >= max_results: 
            break
        
    return statuses[0:len_results]


# STREAMER
# credentials: "searcher_nv_1"
########################################################
def twitter_stream(HASHTAG, len_results):

    CONSUMER_KEY = 'KKpL9KrfKvVk6GXTXZVQGMQZW'
    CONSUMER_SECRET = 'ABikN5quxusBWsWlbxAyMfKEhUQ9AsP3JJXJm6OM9vBXYU15WL'
    OAUTH_TOKEN = '4061210361-nXNyn9jSUimzsGMzSW0RuG8vrDjNjAwXeA3sJGf'
    OAUTH_TOKEN_SECRET = 'NWf3DvEEL1UeogUjoMai4sQGEHkNYu9MpLuY1xezCQUql'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
        CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)

    twitter_stream = twitter.TwitterStream(auth=twitter_api.auth)
    
    print >> sys.stderr, 'Succesfully connected to Twitter STREAMING API ... ... '
    print >> sys.stderr, 'Streming for \'%s\' ... ... ' %HASHTAG
    
    stream = twitter_stream.statuses.filter(track=HASHTAG, language='en')
    print >> sys.stderr, 'Streaming the remaining \t %i \t results ... ... ' %len_results

    statuses = []
    
    for status in stream:
        print status['text']
        print
        statuses.append(status)
        
        if len(statuses) >= len_results:
            break
            
    return statuses

####################################################################################################
# SEARCH AND STREAM
def search_and_stream(HASHTAG, len_results):
    
    statuses_search = twitter_search(HASHTAG, len_results)

#     if len(statuses_search) < len_results:
#         to_stream = len_results-len(statuses_search)
#         stream = twitter_stream(HASHTAG, to_stream)
#         statuses_search += stream
#     else:
#         print 'Found enough results, not necessary to start the stream.'

    return statuses_search

####################################################################################################

def save_track_to_mongo(statuses, track_string, db, coll):
    
    client = pymongo.MongoClient()
    db = client.db
    collection = db.coll
    
    for tweet_doc in statuses:
        try:
            # 2 small modificatio to the tweet as it is gathered from API
            # 1. Add the tags of the search, to know from which search query the document has been obtained
            tweet_doc['track'] = track_string  
            # 2. Substitue the defoult _id assigned by Mongo with the tweet id to prevent the same tweet to be inserted more than once
            tweet_doc['_id'] = tweet_doc["id"]
            del tweet_doc['id']
            mongo_db_coll.insert_one(tweet_doc)
            print "Saving tweet in track %s" %track_string
        except:
            print "Error message from MongoDB, skipping tweet and continuing."
            
def load_from_mongo(mongo_db, mongo_db_coll, return_cursor=False,
                    criteria=None, projection=None, **mongo_conn_kw):
    
    client = pymongo.MongoClient(**mongo_conn_kw)    
    db = pymongo.database.Database(client, mongo_db)
    coll = db.get_collection(mongo_db_coll)

    if criteria is None:
        criteria = {}
    
    if projection is None:
        cursor = coll.find(criteria)
    else:
        cursor = coll.find(criteria, projection)
    
    if return_cursor:
        return cursor
    else:
        return [ item for item in cursor ]    

def get_hashtags_from_collection(cursor_object):
    hashtags = []
    
    result = {"hashtags": hashtags}

    for tweet in cursor_object:

        for hashtag in tweet['entities']['hashtags']:
            hashtags.append(hashtag['text'].lower().encode("utf-8"))
    
    return(result)

def extract_entities_from_collection(cursor_object):
    words = []
    screen_names = []
    hashtags = []
    
    result = {"words": words, "screen_names": screen_names, "hashtags": hashtags}

    for tweet in cursor_object:
        
        wordsVector = getWordsVector(tweet)
        for word in wordsVector:
            words.append(word)

        for user_mention in tweet['entities']['user_mentions']: 
            screen_names.append(user_mention['screen_name'])

        for hashtag in tweet['entities']['hashtags']:
            hashtags.append(hashtag['text'].lower().encode("utf-8"))
    
#     result['words'] = Counter(result['words']).most_common()[:10000]
#     result['screen_names'] = Counter(result['screen_names']).most_common()[:10000]
#     result['hashtags'] = Counter(result['hashtags']).most_common()[:10000]
    return(result)

def processTweet(tweet):
    #Convert to lower case
    tweet = tweet['text'].lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert retweet
    tweet = re.sub('(rt\s)@[^\s]+','RETWEET',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet


def processWord(w):
    #look for 2 or more repetitions of character in a word and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    w = pattern.sub(r"\1\1", w)
    #strip punctuation
    w = w.strip('\'"?,.')
    #check if the word starts with an alphabet
    val = re.search(r"^[a-zA-Z][a-zA-Z0-9-]*$", w)
    if val is None:
        w = 'ABC'
    return w


def getStopWordList(stopWordListFileName):
    st = open(stopWordListFileName, 'r')
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')
    stopWords.append('RETWEET')
    stopWords.append('ABC')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords


def getWordsVector(tweet):
    # initialize vector
    wordsVector = []
    #initialize stopWords
    stopWords = getStopWordList('/Users/nicolavitale/Desktop/twitter_data_analysis/develop/data/stopwords.txt')
    #process tweet and split into words
    tweet = processTweet(tweet)
    words = tweet.split()
    for w in words:
        w = processWord(w)
        if w in stopWords:
            continue
        else:
            wordsVector.append(w.lower())
    return wordsVector


In [3]:
track0_hashtags_list = ['#ecig', '#vaping', '#vape', '#vaporizer', '#marijuana', '#sorrynotsorry', '#eliquid', '#ecigs', '#vapeporn', '#cannabis']
track0_statuses = []

for tag in track0_hashtags_list:

    statuses = search_and_stream(tag, 500)
    track0_statuses += statuses
    
print "TOTAL number of statuses sampled:%i"%len(track0_statuses)

Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#ecig' ... ... 


Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vaping' ... ... 


Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vape' ... ... 


...

Found 	 196 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vapeporn' ... ... 


Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#cannabis' ... ... 


Found 	 200 	 statuses.
TOTAL number of statuses sampled:1994


In [4]:
client = pymongo.MongoClient()
db = client.snowball
collection = db.net

for tweet_doc in track0_statuses:
    try:
        tweet_doc['track'] = 'track0'
        tweet_doc['_id'] = tweet_doc["id"]
        del tweet_doc['id']
        collection.insert_one(tweet_doc)
        print "Saving tweet."
    except:
        print "Preventing duplicate, skipping tweet and continuing."

Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving

In [5]:
df = pd.read_csv('~/Desktop/twitter_data_analysis/develop/Py/track1.csv')
track1_list = df['x'].tolist()
track1_list_hashed = []
for h in track1_list:
    h = '#'+h
    track1_list_hashed.append(h)
    
print track1_list_hashed

['#vapeon', '#ecigarette', '#vapor', '#atomizers', '#cbdvape', '#dabs', '#oilvape', '#vapepen', '#waxvape', '#legalizeit', '#mme', '#ca', '#selfie', '#40likes', '#class', '#country', '#quoteoftheday', '#sand', '#teen', '#5sosweekday2calum', '#apollomod', '#aspire', '#joyetech', '#modz', '#smoke', '#v2', '#vapejuice', '#reddit', '#mmj', '#investsmarter']


In [6]:
print track1_list_hashed
track1_statuses = []

for tag in track1_list_hashed:

    statuses = search_and_stream(tag, 5000)
    track1_statuses += statuses
    
print "TOTAL number of statuses sampled:%i"%len(track1_statuses)

['#vapeon', '#ecigarette', '#vapor', '#atomizers', '#cbdvape', '#dabs', '#oilvape', '#vapepen', '#waxvape', '#legalizeit', '#mme', '#ca', '#selfie', '#40likes', '#class', '#country', '#quoteoftheday', '#sand', '#teen', '#5sosweekday2calum', '#apollomod', '#aspire', '#joyetech', '#modz', '#smoke', '#v2', '#vapejuice', '#reddit', '#mmj', '#investsmarter']


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vapeon' ... ... 


Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#ecigarette' ... ... 


Found 	 200 	 statuses.


...

Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#mmj' ... ... 


Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#investsmarter' ... ... 


Found 	 139 	 statuses.
TOTAL number of statuses sampled:3985


In [7]:
for tweet_doc in track1_statuses:
    try:
        tweet_doc['track'] = 'track1'
        tweet_doc['_id'] = tweet_doc["id"]
        del tweet_doc['id']
        collection.insert_one(tweet_doc)
        print "Saving tweet."
    except:
        print "Preventing duplicate, skipping tweet and continuing."

Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Saving tweet.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
S

In [8]:
df = pd.read_csv('~/Desktop/twitter_data_analysis/develop/Py/track2.csv')
track2_list = df['x'].tolist()
track2_list_hashed = []
for h in track2_list:
    h = '#'+h
    track2_list_hashed.append(h)
    
print track2_list_hashed

['#vapelife', '#notblowingsmoke', '#weed', '#icantbeurfriendbecause', '#sept13', '#ejuice', '#vapefam', '#vapestoreworldwide', '#apx', '#pulsar', '#wax', '#sidekick', '#ptsd', '#darkhair', '#old', '#safetyfirst', '#seatbelt', '#ggss', '#pakganern', '#internet']


In [9]:
print track2_list_hashed
track2_statuses = []

for tag in track2_list_hashed:

    statuses = search_and_stream(tag, 5000)
    track2_statuses += statuses
    
print "TOTAL number of statuses sampled:%i"%len(track2_statuses)

['#vapelife', '#notblowingsmoke', '#weed', '#icantbeurfriendbecause', '#sept13', '#ejuice', '#vapefam', '#vapestoreworldwide', '#apx', '#pulsar', '#wax', '#sidekick', '#ptsd', '#darkhair', '#old', '#safetyfirst', '#seatbelt', '#ggss', '#pakganern', '#internet']


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vapelife' ... ... 


Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#notblowingsmoke' ... ... 


Found 	 200 	 statuses.


...

Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#pakganern' ... ... 


Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#internet' ... ... 


Found 	 93 	 statuses.
TOTAL number of statuses sampled:3374


In [11]:
for tweet_doc in track2_statuses:
    try:
        tweet_doc['track'] = 'track2'
        tweet_doc['_id'] = tweet_doc["id"]
        del tweet_doc['id']
        collection.insert_one(tweet_doc)
        print "Saving tweet."
    except:
        print "Preventing duplicate, skipping tweet and continuing."

Saving tweet.
Saving tweet.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skip

In [12]:
df = pd.read_csv('~/Desktop/twitter_data_analysis/develop/Py/track3.csv')
track3_list = df['x'].tolist()
track3_list_hashed = []
for h in track3_list:
    h = '#'+h
    track3_list_hashed.append(h)

print track3_list_hashed

['#oregon', '#or', '#veteran', '#jobs', '#jobsearch', '#beach', '#itsagreatdayfor', '#5sosweek', '#5sosaregreatbecause', '#all5sosmembersareimportant', '#mtvhottest', '#veranomtv2016', '#5sosfanswantkeekstocomeback', '#a\xc5\x9fk', '#esigara', '#likit', '#nikotin', '#saglik', '#sigara', '#cbd', '#thc', '#crowdfunding', '#growthindustries', '#hillarysemails', '#feartwd', '#besteliquid', '#did', '#mentalhealth', '#tech', '#howto', '#internautday', '#question']


In [13]:
print track3_list_hashed
track3_statuses = []

for tag in track3_list_hashed:

    statuses = search_and_stream(tag, 5000)
    track3_statuses += statuses
    
print "TOTAL number of statuses sampled:%i"%len(track3_statuses)

['#oregon', '#or', '#veteran', '#jobs', '#jobsearch', '#beach', '#itsagreatdayfor', '#5sosweek', '#5sosaregreatbecause', '#all5sosmembersareimportant', '#mtvhottest', '#veranomtv2016', '#5sosfanswantkeekstocomeback', '#a\xc5\x9fk', '#esigara', '#likit', '#nikotin', '#saglik', '#sigara', '#cbd', '#thc', '#crowdfunding', '#growthindustries', '#hillarysemails', '#feartwd', '#besteliquid', '#did', '#mentalhealth', '#tech', '#howto', '#internautday', '#question']


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#oregon' ... ... 


Found 	 93 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#or' ... ... 


Found 	 96 	 statuses.


...

Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#growthindustries' ... ... 


Found 	 10 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#hillarysemails' ... ... 


TwitterHTTPError: Twitter sent status 429 for URL: 1.1/search/tweets.json using parameters: (count=100&include_entities=1&language=en&max_id=768803533235875839&oauth_consumer_key=cOExq4T3uYu9CbdqmMUukGVqV&oauth_nonce=15331969862966170376&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1472134652&oauth_token=4061210361-uHZhAHfTim8nnyyCWeZXMfLbypbq3Z2jgGcTmhu&oauth_version=1.0&q=%252523hillarysemails&oauth_signature=yzinoC2jSgDHhmKxfn%2BWaNIUNZM%3D)
details: {u'errors': [{u'message': u'Rate limit exceeded', u'code': 88}]}

In [14]:
for tweet_doc in track3_statuses:
    try:
        tweet_doc['track'] = 'track3'
        tweet_doc['_id'] = tweet_doc["id"]
        del tweet_doc['id']
        collection.insert_one(tweet_doc)
        print "Saving tweet."
    except:
        print "Preventing duplicate, skipping tweet and continuing."

Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
S

In [17]:
df = pd.read_csv('~/Desktop/twitter_data_analysis/develop/Py/track4.csv')
track4_list = df['x'].tolist()
track4_list_hashed = []
for h in track4_list:
    h = '#'+h
    track4_list_hashed.append(h)

print track4_list_hashed

['#tgod', '#blog', '#inspire', '#alters', '#selfcare', '#sicknotweak', '#stigma', '#trauma', '#job', '#hiring', '#portland', '#5sosweekday1luke', '#all5sosmembersmatter', '#fifthharmony', '#mpn', '#startup', '#gadgets', '#entrepreneurs', '#score', '#hillaryshealth', '#clintonfoundation', '#thewalkingdead', '#info', '#olympics', '#clexa', '#fearthewalkingdead', '#mods', '#bogo', '#keylimepievape', '#abuse', '#news', '#how', '#worldwideweb']


In [18]:
print track4_list_hashed
track4_statuses = []

for tag in track4_list_hashed:

    statuses = search_and_stream(tag, 5000)
    track4_statuses += statuses
    
print "TOTAL number of statuses sampled:%i"%len(track4_statuses)

['#tgod', '#blog', '#inspire', '#alters', '#selfcare', '#sicknotweak', '#stigma', '#trauma', '#job', '#hiring', '#portland', '#5sosweekday1luke', '#all5sosmembersmatter', '#fifthharmony', '#mpn', '#startup', '#gadgets', '#entrepreneurs', '#score', '#hillaryshealth', '#clintonfoundation', '#thewalkingdead', '#info', '#olympics', '#clexa', '#fearthewalkingdead', '#mods', '#bogo', '#keylimepievape', '#abuse', '#news', '#how', '#worldwideweb']


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#tgod' ... ... 


Found 	 200 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#blog' ... ... 


Found 	 97 	 statuses.


...

Found 	 68 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#how' ... ... 


Found 	 57 	 statuses.


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#worldwideweb' ... ... 


Found 	 200 	 statuses.
TOTAL number of statuses sampled:5174


In [19]:
for tweet_doc in track4_statuses:
    try:
        tweet_doc['track'] = 'track4'
        tweet_doc['_id'] = tweet_doc["id"]
        del tweet_doc['id']
        collection.insert_one(tweet_doc)
        print "Saving tweet."
    except:
        print "Preventing duplicate, skipping tweet and continuing."

Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving